# Parameter tuning for regression models

## GridSearchCV

In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_percentage_error


### Setting up


In [20]:
# Load data
df = pd.read_excel("./housing_data.xlsx")

# Extract data 
X = df[["LSTAT"]].values
y = df["MEDV"].values


# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# Standardize data
sc = StandardScaler()
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)

# Classifier
svr = SVR()

In [21]:
df

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273,21.0,393.45,6.48,22.0


In [22]:
pd.DataFrame(data=svr.get_params(), index=["param"]).T

,param
C,1.0
cache_size,200
coef0,0.0
degree,3
epsilon,0.1
gamma,scale
kernel,rbf
max_iter,-1
shrinking,True
tol,0.001


### Define parameter sets

In [23]:
param_range = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]

set1 = {'C': param_range,
        'kernel': ['linear']}

set2 = {'C': param_range,
        'gamma': param_range,
        'kernel': ['rbf']}

param_grid = [set1, set2]

### GridSearchCV classifier

- Don't forget to change the scoring function.

In [24]:
gs = GridSearchCV(estimator=svr, 
                  param_grid=param_grid, 
                  scoring='neg_mean_squared_error', 
                  cv=5,
                  n_jobs=-1)

In [25]:
# Get parameter names
for k, v in gs.get_params().items():
    print(f"{k:35.35s}: {str(v)}")

cv                                 : 5
error_score                        : nan
estimator__C                       : 1.0
estimator__cache_size              : 200
estimator__coef0                   : 0.0
estimator__degree                  : 3
estimator__epsilon                 : 0.1
estimator__gamma                   : scale
estimator__kernel                  : rbf
estimator__max_iter                : -1
estimator__shrinking               : True
estimator__tol                     : 0.001
estimator__verbose                 : False
estimator                          : SVR()
n_jobs                             : -1
param_grid                         : [{'C': [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0], 'kernel': ['linear']}, {'C': [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0], 'gamma': [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0], 'kernel': ['rbf']}]
pre_dispatch                       : 2*n_jobs
refit                              : True
return_train_score            

### Training

In [26]:
gs.fit(X_train_std,y_train)

GridSearchCV(cv=5, estimator=SVR(), n_jobs=-1,
             param_grid=[{'C': [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0,
                                1000.0],
                          'kernel': ['linear']},
                         {'C': [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0,
                                1000.0],
                          'gamma': [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0,
                                    1000.0],
                          'kernel': ['rbf']}],
             scoring='neg_mean_squared_error')

In [27]:
df = pd.DataFrame(gs.cv_results_)
print(df.shape)
display(df.head())

(72, 16)


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,param_gamma,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.004210,0.000290,0.001096,7.713961e-05,0.0001,linear,NaN,"{'C': 0.0001, 'kernel': 'linear'}",-72.396781,-58.119588,-86.219069,-130.861376,-84.919782,-86.503319,24.392032,56
1,0.004122,0.000425,0.001046,2.098083e-06,0.001,linear,NaN,"{'C': 0.001, 'kernel': 'linear'}",-70.306888,-56.284045,-83.866296,-127.823041,-82.414877,-84.139030,23.994033,47
2,0.003778,0.000223,0.001042,2.153119e-05,0.01,linear,NaN,"{'C': 0.01, 'kernel': 'linear'}",-55.996471,-44.417588,-68.228734,-106.866318,-64.608844,-68.023591,21.082481,37
3,0.003619,0.000503,0.000999,7.478899e-07,0.1,linear,NaN,"{'C': 0.1, 'kernel': 'linear'}",-37.913040,-32.934327,-44.492693,-77.335299,-39.907262,-46.516524,15.849739,28
4,0.005109,0.000736,0.000704,4.009440e-04,1.0,linear,NaN,"{'C': 1.0, 'kernel': 'linear'}",-34.684879,-32.234355,-38.762849,-68.796371,-34.055135,-41.706718,13.711997,22


In [28]:
df = df.sort_values(by=['rank_test_score'])
display(df.head())

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,param_gamma,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
67,0.010915,0.000488,0.003592,0.001047,1000.0,rbf,0.1,"{'C': 1000.0, 'gamma': 0.1, 'kernel': 'rbf'}",-24.955601,-22.565049,-25.035532,-50.304312,-21.284258,-28.828950,10.832755,1
53,0.006614,0.000663,0.003481,0.000448,10.0,rbf,10.0,"{'C': 10.0, 'gamma': 10.0, 'kernel': 'rbf'}",-24.694972,-21.360634,-26.851402,-49.551585,-22.955535,-29.082826,10.395947,2
52,0.011902,0.012559,0.003587,0.000356,10.0,rbf,1.0,"{'C': 10.0, 'gamma': 1.0, 'kernel': 'rbf'}",-26.048003,-22.190452,-24.707081,-51.087331,-22.382947,-29.283163,10.997710,3
60,0.014935,0.013839,0.004125,0.000919,100.0,rbf,1.0,"{'C': 100.0, 'gamma': 1.0, 'kernel': 'rbf'}",-25.613342,-22.102335,-29.149374,-49.933217,-21.201774,-29.600008,10.547809,4
61,0.011910,0.003766,0.006609,0.005833,100.0,rbf,10.0,"{'C': 100.0, 'gamma': 10.0, 'kernel': 'rbf'}",-24.740151,-22.495047,-28.276952,-50.295533,-24.060930,-29.973723,10.335734,5


### Refitting
- Note that grid search already refit the entire training data with the best parameters. You can check this from this setting.

In [29]:
gs.refit

True

In [30]:
print(gs.best_score_)
print(gs.best_params_)

-28.828950253331676
{'C': 1000.0, 'gamma': 0.1, 'kernel': 'rbf'}


In [31]:
y_pred = gs.predict(X_test_std)
print(mean_absolute_percentage_error(y_test, y_pred))

0.1781562895108512
